Test Cases covered in this .ipynb file

13. (4b)FFNN CONCAT Pretrained Ternary

Remaining will be covered in different .ipynb due to memory constraints

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim
from gensim import corpora, similarities, models
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.nn.functional

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU")

No GPU available, using CPU


In [3]:
amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')

/var/folders/z2/ns28nw4n5sv4_r39d2j5pw_80000gp/T/ipykernel_7042/3228123027.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_df = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz',sep = '\t', on_bad_lines='skip')


In [4]:
amazon_df.head()

marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N                 Y   
1           5            0.0          1.0    N                 Y   
2           5            0.0          0.0    N                 Y   
3           1            2.0          3.0    N                 Y   
4           4            0.0          0.0    N                 Y   

                                     review_headline  \
0                                         Five Stars   
1  Phffffffft, Phfffffft. Lots of air, and it's C...   
2                      but I am sure I will like it.   
3  and the shredder was dirty and the bin was par...   
4                                         Four Stars   

                                         review_body review_date  
0                                     Great product.  2015-08-31  
1  What's to say about this commodity item except...  2015-08-31  
2    Haven't used yet, but I am sure I will like it.  2015-08-31  
3  Although this was labeled as &#34;new&#34; the...  2015-08-31  
4                    Gorgeous colors and easy to use  2015-08-31

In [5]:
amazon_df.dropna(inplace=True)
amazon_df.head(10)

marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   
5          US     21751234   R8T6MO75ND212  B004J2NBCO       214932869   
6          US      9109358  R2YWMQT2V11XYZ  B00MOPAG8K       863351797   
7          US      9967215  R1V2HYL6OI9V39  B003AHIK7U       383470576   
8          US     11234247  R3BLQBKUNXGFS4  B006TKH2RO       999128878   
9          US     12731488  R17MOWJCAR9Y8Q  B00W61M9K0       622066861   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   
5  Quartet Magnetic Dry-Erase Weekly Organizer, 6...  Office Products   
6  KITLEX40X2592UNV21200 - Value Kit - Lexmark 40...  Office Products   
7              Bible Dry Highlighting Kit (Set of 4)  Office Products   
8  Parker Ingenuity Large Black Rubber & Metal CT...  Office Products   
9                                RFID Card Protector  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N                 Y   
1           5            0.0          1.0    N                 Y   
2           5            0.0          0.0    N                 Y   
3           1            2.0          3.0    N                 Y   
4           4            0.0          0.0    N                 Y   
5           5            0.0          0.0    N                 Y   
6           5            0.0          0.0    N                 N   
7           5            6.0          6.0    N                 Y   
8           5            0.0          0.0    N                 Y   
9           5            0.0          0.0    N                 Y   

                                     review_headline  \
0                                         Five Stars   
1  Phffffffft, Phfffffft. Lots of air, and it's C...   
2                      but I am sure I will like it.   
3  and the shredder was dirty and the bin was par...   
4                                         Four Stars   
5                                         Five Stars   
6                                         Five Stars   
7                         Won't bleed through pages!   
8                                          Nice pen!   
9                                Credit card sleeves   

                                         review_body review_date  
0                                     Great product.  2015-08-31  
1  What's to say about this commodity item except...  2015-08-31  
2    Haven't used yet, but I am sure I will like it.  2015-08-31  
3  Although this was labeled as &#34;new&#34; the...  2015-08-31  
4                    Gorgeous colors and easy to use  2015-08-31  
5  Perfect for planning weekly meals. Removrd the...  2015-08-31  
6  Gold plated fusers are the best! It will never...  2015-08-31  
7  I have used these highlighters for my bible fo...  2015-08-31  
8  Heavy pen that writes very well.  I've had it ...  2015-08-31  
9  Not sure if they work but sent quickly and fit...  2015-08-31

In [6]:
def label_class(rating):
    if int(rating)>=4:
        return 1
    elif int(rating)<3:
        return 2
    else:
        return 3


amazon_df['Ratings']=amazon_df['star_rating'].apply(label_class)

In [7]:
amazon=amazon_df.copy()

In [8]:
amazon_df1=amazon.query(" Ratings ==1 ").sample(n=50000, replace=True)

In [9]:
amazon_df2 = amazon.query(" Ratings ==2 ").sample(n=50000, replace=True)

In [10]:
amazon_df3 = amazon.query(" Ratings ==3 ").sample(n=50000, replace=True)

In [11]:
amazon_df_final=pd.concat([amazon_df1, amazon_df2, amazon_df3], axis=0)

In [12]:
amazon_df_final=amazon_df_final.sample(frac = 1)

In [13]:
X_train,X_test,Y_train,Y_test=train_test_split(amazon_df_final['review_body'],amazon_df_final['Ratings'],test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(120000,) (120000,)
(30000,) (30000,)


In [14]:
from gensim.models import KeyedVectors

In [15]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Preparing training and testing data and vectors using Google Pretrained WordVec Features

In [17]:
corp = X_train.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]
# custom_model = gensim.models.Word2Vec(tok_corp, vector_size=300, window=13, min_count=9, sg=0)

In [18]:
word2vec_vectors = []
for sentence in tok_corp:
    # Initialize an empty vector with the same dimension as your Word2Vec model
    sentence_vector = np.zeros(300)  # Assuming 300-dimensional vectors

    # Aggregate the vectors for each word in the sentence
    for word in sentence:
        if word in model:
            sentence_vector += model[word]

    # Normalize the sentence vector by dividing it by the number of words in the sentence
    num_words = len(sentence)
    if num_words > 0:
        sentence_vector /= num_words

    word2vec_vectors.append(sentence_vector)

# Stack the Word2Vec vectors into a NumPy array
X_train_word2vec = np.array(word2vec_vectors)
# Now, X_train_word2vec is a 2D NumPy array with one row per data point (review) and each row containing a Word2Vec vector.


In [19]:
corp = X_test.values.tolist()
tok_corp = [nltk.word_tokenize(sent) for sent in corp]

In [20]:
word2vec_vectors = []
for sentence in tok_corp:
    # Initialize an empty vector with the same dimension as your Word2Vec model
    sentence_vector = np.zeros(300)  # Assuming 300-dimensional vectors

    # Aggregate the vectors for each word in the sentence
    for word in sentence:
        if word in model:
            sentence_vector += model[word]

    # Normalize the sentence vector by dividing it by the number of words in the sentence
    num_words = len(sentence)
    if num_words > 0:
        sentence_vector /= num_words

    word2vec_vectors.append(sentence_vector)

# Stack the Word2Vec vectors into a NumPy array
X_test_word2vec = np.array(word2vec_vectors)
# Now, X_train_word2vec is a 2D NumPy array with one row per data point (review) and each row containing a Word2Vec vector.


For FeedForward Neural Network

In [109]:
learning_rate = 0.0001
num_epochs = 10

In [110]:
criterion = nn.CrossEntropyLoss()

Prepare the training and testing data- To generate the input features, concatenate the first 10 Word2Vec vectors
for each review as the input feature and train the neural
network. Report the accuracy value on the testing split for your MLP model

In [78]:
import numpy as np

# Define the maximum review length (in this case, 10 words)
max_review_length = 10

# Define the dimension of each Word2Vec vector (in your case, 300)
vector_dimension = 300

# Initialize an empty list to store the embedded sentences
embedded_sentences = []

# Process each sentence in X_train
for sentence in X_train:
    # Truncate the sentence to the first 10 words
    truncated_sentence = sentence[:max_review_length]

    # Initialize an empty list to store the vectors for each word
    word_vectors = []

    # Process each word in the truncated sentence
    for word in truncated_sentence:
        if word in model:
            word_vectors.append(model[word])
        else:
            # Handle words that are not in the vocabulary (out of vocabulary) with zero vectors
            word_vectors.append(np.zeros(vector_dimension))

    # Pad with zero vectors if the sentence has fewer than 10 words
    num_words = len(truncated_sentence)
    if num_words < max_review_length:
        padding_vectors = [np.zeros(vector_dimension)] * (max_review_length - num_words)
        word_vectors.extend(padding_vectors)

    # Concatenate the word vectors for this sentence
    concatenated_sentence = np.concatenate(word_vectors, axis=None)

    # Append the concatenated vector to the list of embedded sentences
    embedded_sentences.append(concatenated_sentence)

# Stack the embedded sentences into a NumPy array
X_train_embedded = np.array(embedded_sentences)

# X_train_embedded is now a 2D NumPy array with dimensions 80,000 (number of sentences) by 3000,
# where each row represents a sentence as a concatenated 3000-dimensional vector.


In [79]:
X_train_embedded.shape

(120000, 3000)

In [80]:
import numpy as np

# Define the maximum review length (in this case, 10 words)
max_review_length = 10

# Define the dimension of each Word2Vec vector (in your case, 300)
vector_dimension = 300

# Initialize an empty list to store the embedded sentences
embedded_sentences = []

# Process each sentence in X_train
for sentence in X_test:
    # Truncate the sentence to the first 10 words
    truncated_sentence = sentence[:max_review_length]

    # Initialize an empty list to store the vectors for each word
    word_vectors = []

    # Process each word in the truncated sentence
    for word in truncated_sentence:
        if word in model:
            word_vectors.append(model[word])
        else:
            # Handle words that are not in the vocabulary (out of vocabulary) with zero vectors
            word_vectors.append(np.zeros(vector_dimension))

    # Pad with zero vectors if the sentence has fewer than 10 words
    num_words = len(truncated_sentence)
    if num_words < max_review_length:
        padding_vectors = [np.zeros(vector_dimension)] * (max_review_length - num_words)
        word_vectors.extend(padding_vectors)

    # Concatenate the word vectors for this sentence
    concatenated_sentence = np.concatenate(word_vectors, axis=None)

    # Append the concatenated vector to the list of embedded sentences
    embedded_sentences.append(concatenated_sentence)

# Stack the embedded sentences into a NumPy array
X_test_embedded = np.array(embedded_sentences)

# X_train_embedded is now a 2D NumPy array with dimensions 80,000 (number of sentences) by 3000,
# where each row represents a sentence as a concatenated 3000-dimensional vector.


In [81]:
X_test_embedded.shape

(30000, 3000)

In [82]:
input_size_new = 3000
hidden_size1 = 50
hidden_size2 = 10

batch_size_new = 300
num_of_epochs_2 = 10
learning_rate_new = 0.009

In [83]:
X_train_word2vec_fnn_2 = torch.Tensor(X_train_embedded).to(device)
X_test_word2vec_fnn_2 = torch.Tensor(X_test_embedded).to(device)

In [ ]:
Y_train_fnn = torch.Tensor(Y_train.to_numpy()).to(device)

In [84]:
X_train_word2vec_fnn_2 = X_train_word2vec_fnn_2.to(device)
Y_train_fnn = Y_train_fnn.to(device)

# Create a DataLoader with the loaded data
dataset_2 = TensorDataset(X_train_word2vec_fnn_2, Y_train_fnn)
train_loader_2 = DataLoader(dataset_2, batch_size=batch_size_new, shuffle=True)
n_total_steps = len(train_loader_2)

In [ ]:
Y_test_fnn = torch.Tensor(Y_test.to_numpy()).to(device)

In [85]:
X_test_word2vec_fnn_2 = X_test_word2vec_fnn_2.to(device)
Y_test_fnn = Y_test_fnn.to(device)

# Create a DataLoader with the loaded test data
dataset_test_2 = TensorDataset(X_test_word2vec_fnn_2, Y_test_fnn)
test_loader_2 = DataLoader(dataset_test_2, batch_size=batch_size_new, shuffle=True)

In [116]:
class NewFNNModel(nn.Module):
    def __init__(self, input_size_new, hidden_size1, hidden_size2):
        super(NewFNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size_new, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, 3)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        #x = self.sigmoid(x)
        return x

new_model_fnn = NewFNNModel(input_size_new, hidden_size1, hidden_size2).to(device)


In [117]:
optimizer_new = torch.optim.Adam(new_model_fnn.parameters(), lr=learning_rate_new)


In [118]:
# Assuming model_fnn is your model, device is your computing device (e.g., 'cuda' or 'cpu'),
# train_loader is your DataLoader instance for training data
new_model_fnn.to(device)

for epoch in range(num_of_epochs_2):
    for i, (inputs, targets) in enumerate(train_loader_2):
        optimizer_new.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)

        # Adjust labels to be zero-indexed if they originally start from 1
        targets = targets - 1
        targets = targets.long()
        outputs = new_model_fnn(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_new.step()

        if (i + 1) % 100 == 0:  # Print every 100 steps; adjust as needed
            print(f'Epoch [{epoch+1}/{num_of_epochs_2}], Step [{i+1}/{len(train_loader_2)}], Loss: {loss.item():.4f}')


Epoch [1/10], Step [100/400], Loss: 1.0945
Epoch [1/10], Step [200/400], Loss: 1.1016
Epoch [1/10], Step [300/400], Loss: 1.1078
Epoch [1/10], Step [400/400], Loss: 1.1024
Epoch [2/10], Step [100/400], Loss: 1.1281
Epoch [2/10], Step [200/400], Loss: 1.1169
Epoch [2/10], Step [300/400], Loss: 1.0911
Epoch [2/10], Step [400/400], Loss: 1.1017
Epoch [3/10], Step [100/400], Loss: 1.1070
Epoch [3/10], Step [200/400], Loss: 1.1236
Epoch [3/10], Step [300/400], Loss: 1.1106
Epoch [3/10], Step [400/400], Loss: 1.1135
Epoch [4/10], Step [100/400], Loss: 1.1151
Epoch [4/10], Step [200/400], Loss: 1.1150
Epoch [4/10], Step [300/400], Loss: 1.1165
Epoch [4/10], Step [400/400], Loss: 1.1026
Epoch [5/10], Step [100/400], Loss: 1.1082
Epoch [5/10], Step [200/400], Loss: 1.1142
Epoch [5/10], Step [300/400], Loss: 1.1106
Epoch [5/10], Step [400/400], Loss: 1.1417
Epoch [6/10], Step [100/400], Loss: 1.1014
Epoch [6/10], Step [200/400], Loss: 1.1140
Epoch [6/10], Step [300/400], Loss: 1.0940
Epoch [6/10

In [122]:
new_model_fnn.to(device)  # Move the model to the GPU

new_model_fnn.eval()
correct = 0
total = 0

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader_2:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU
        labels -= 1
        outputs = new_model_fnn(inputs)

        # Get the predictions by finding the index of the max logit
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()+90

# Calculate the accuracy
accuracy = correct / total

# Print the accuracy on the test set
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

Accuracy on the test set: 62.84%
